In [ ]:
import tweepy
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import datetime
from datetime import timedelta
import sqlite3


def update_data():

    token_data = open(r"C:\Users\Rengga\Downloads\secrettoken.csv",
                      encoding='utf-8-sig')
    tokens = csv.reader(token_data, delimiter=',')

    data_token = []
    for row in tokens:
        data_token.append(row[0])

    consumer_key = data_token[0]
    consumer_secret = data_token[1]
    access_token = data_token[2]
    access_token_secret = data_token[3]

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    date_entry = input('Masukan tanggal dengan format YYYY-MM-DD :')
    year, month, day = map(int, date_entry.split('-'))
    date1 = datetime.date(year, month, day)

    hari = date1 - timedelta(days=31)
    hari = hari.strftime("%Y-%m-%d")

    search_words = "Telkomsel"
    date_since = hari
    new_search = search_words + " -filter:retweets"

    tweets = tweepy.Cursor(api.search,
                           q=new_search,
                           lang="id",
                           since=date_since, until=date1).items(1000)
    user = []
    time = []
    items = []
    for tweet in tweets:
        time.append(tweet.created_at.date())
        user.append("@%s" % (tweet.user.screen_name))
        items.append(' '.join(re.sub(
            "(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet.text).split()))

    df = pd.DataFrame({'tweet': items, 'time': time, 'user': user})

    df.to_csv(r"C:\Users\Rengga\Downloads\tweet.csv", index=False)

    class basisdata():
        def __init__(self):
            pass

        def open_connect(self, basisdata):
            connection = sqlite3.connect(basisdata)
            return connection

        def create_table(self, connection):
            query = 'create table if not exists tweet(id integer primary key autoincrement, tweet text not null);'
            cursor = connection.cursor()
            cursor.execute(query)
            cursor.close

        def insert_data(self, connection):
            query = '''INSERT INTO tweet (tweet, tanggal, nama) values (?,?,?)'''
            cursor = connection.cursor()
            with open(r"C:\Users\Rengga\Downloads\tweet.csv") as csvfile:
                csv_reader = csv.reader(csvfile)
                next(csv_reader)
                for row in csv_reader:
                    tweet = row[0]
                    tanggal = row[1]
                    nama = row[2]
                    cursor.execute(query, [tweet, tanggal, nama])
                    cursor.close

        def remove_duplicate(self, connection):
            query = '''SELECT  DISTINCT tweet,tanggal,nama FROM tweet ORDER BY idtweet'''
            cursor = connection.cursor()
            cursor.execute(query)
            cursor.close

        def close_connect(self, connection):
            connection.commit()
            connection.close()

    data = basisdata()
    connection = data.open_connect(
        r"C:\Users\Rengga\Downloads\renggadhamar06.db")
    # data.create_table(connection)
    data.insert_data(connection)
    data.remove_duplicate(connection)
    data.close_connect(connection)

    print('Data Telah Terupdate!!')
    print('===================================')


def lihat_data():

    date_entry_ = input(
        'Masukan tanggal yang akan dipilih dengan format YYYY-MM-DD :')
    year_, month_, day_ = map(int, date_entry_.split('-'))
    date_ = datetime.date(year_, month_, day_)

    date_entry = input(
        'Masukan tanggal akhir yang akan dipilih dengan format YYYY-MM-DD :')
    year, month, day = map(int, date_entry.split('-'))
    date = datetime.date(year, month, day)

    connection = sqlite3.connect(
        r"C:\Users\Rengga\Downloads\renggadhamar06.db")

    query = '''SELECT nama,tanggal,tweet FROM tweet WHERE tanggal BETWEEN ? AND ?'''
    cursor = connection.cursor()
    cursor.execute(query, (date_, date, ))
    hasilsemua = cursor.fetchall()
    cursor.close
    connection.commit()
    connection.close()

    dataframe = pd.DataFrame(hasilsemua, columns=['user', 'tanggal', 'tweet'])
    print(dataframe)


def ukur():

    connection = sqlite3.connect(
        r"C:\Users\Rengga\Downloads\renggadhamar06.db")
    query = '''SELECT idtweet, tweet FROM tweet '''
    cursor = connection.cursor()
    cursor.execute(query)
    hasilsemua = cursor.fetchall()
    cursor.close
    connection.commit()
    connection.close()

    # dataframe = pd.DataFrame(hasilsemua, columns=['tweet'])

    pos_list = open(r"C:\Users\Rengga\Downloads\kata_positif.txt")
    pos_kata = pos_list.readlines()
    neg_list = open(r"C:\Users\Rengga\Downloads\kata_negatif.txt")
    neg_kata = neg_list.readlines()

    cek = []
    k = 0
    for x in hasilsemua:
        row = hasilsemua[k][1]
        k += 1
        cek.append(row)

    id_tweet = []
    j = 0
    for y in hasilsemua:
        row_ = hasilsemua[j][0]
        j += 1
        id_tweet.append(row_)

    S = []
    for item in cek:
        count_p = 0
        count_n = 0
        for kata_pos in pos_kata:
            if kata_pos.strip() in item:
                count_p += 1
        for kata_neg in neg_kata:
            if kata_neg.strip() in item:
                count_n += 1

        S.append(count_p - count_n)

    # print("-----------------------------------------------------")
    z = 0
    while z < len(S):

        connection = sqlite3.connect(
            r"C:\Users\Rengga\Downloads\renggadhamar06.db")
        cursor = connection.cursor()
        sql_update_query = """Update tweet set sentiment = ? where idtweet = ?"""
        data = (S[z], id_tweet[z])
        cursor.execute(sql_update_query, data)
        connection.commit()
        cursor.close()
        connection.close()
        z += 1

    print('Sentiment sudah dihitung!!')
    print('===================================')


def visual():

    date_entry_ = input(
        'Masukan tanggal awal yang akan dipilih dengan format YYYY-MM-DD :')
    year_, month_, day_ = map(int, date_entry_.split('-'))
    date_ = datetime.date(year_, month_, day_)

    date_entry = input(
        'Masukan tanggal akhir yang akan dipilih dengan format YYYY-MM-DD :')
    year, month, day = map(int, date_entry.split('-'))
    date = datetime.date(year, month, day)

    connection = sqlite3.connect(
        r"C:\Users\Rengga\Downloads\renggadhamar06.db")
    query = '''SELECT sentiment FROM tweet WHERE tanggal BETWEEN ? AND ?'''
    cursor = connection.cursor()
    cursor.execute(query, (date_, date, ))
    hasilsemua = cursor.fetchall()
    cursor.close
    connection.commit()
    connection.close()

    dataframe = pd.DataFrame(hasilsemua, columns=['sentiment'])
    print("Nilai rata-rata: "+str(np.mean(dataframe["sentiment"])))
    print("Nilai rata-rata: "+str(np.median(dataframe["sentiment"])))
    print("Standar deviasi: "+str(np.std(dataframe["sentiment"])))

    labels, counts = np.unique(dataframe["sentiment"], return_counts=True)
    plt.bar(labels, counts, align='center')
    plt.gca().set_xticks(labels)
    plt.show()


pilihan = 0
while pilihan != 5:

    print('Apa yang ingin anda lakukan?')
    print('      1. Update data')
    print('      2. Update nilai sentiment')
    print('      3. Lihat data')
    print('      4. Visualisasi')
    print('      5. Keluar')
    print('      Input Anda : ')
    pilihan = int(input('      '))
    print('===================================')
    print('Pilihan Anda =', pilihan)
    if pilihan == 1:
        # print('====Anda akan mengupdate data di database====')
        # print('====Tanggal berapa Tweet yang akan anda Scraping dari Twitter====')
        update_data()
    elif pilihan == 2:
        # print('====Anda akan menghitung Sentiment dari Tweet di Database====')
        ukur()
    elif pilihan == 3:
        # print('====Anda akan melihat tweet di Database====')
        # print('====Tanggal berapa Tweet yang Anda ingin lihat====')
        lihat_data()
    elif pilihan == 4:
        # print('====Anda akan melihat perhitungan dan visualisasi Sentiment====')
        # print('====Tanggal berapa Tweet yang Anda ingin lihat====')
        visual()
    elif pilihan == 5:
        break
    else:
        print('System tidak mengenali, pilih 1 s/d 5')
